In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [3]:
df_train = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/y_val.csv')

df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)

# display(df_train)
# display(df_test)
# display(label_train)
# display(label_test)

In [4]:
# display(label_train)
# display(label_test)
print(label_train["label"].value_counts())

label
3    1089
1     714
2     378
0     202
Name: count, dtype: int64


In [5]:
# display(df_train)
# display(df_test)
# display(label_train)
# display(label_test)
# print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
# print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")

In [7]:
features_ranks_df = pd.read_csv('../datasets/nprint-raw/stream-video-c10-little/feature_importance-c10.csv')
display(features_ranks_df.head(40))

,Unnamed: 0,importance,stddev,p_value,n,p99_high,p99_low
0,pkt_9_ipv4_dst_6,0.0658,0.010809,6.359170e-09,10,0.076909,0.054691
1,pkt_2_rts,0.0248,0.005594,1.012670e-07,10,0.030549,0.019051
2,pkt_6_rts,0.0244,0.006518,4.324374e-07,10,0.031099,0.017701
3,pkt_4_rts,0.0238,0.002741,2.730473e-10,10,0.026617,0.020983
4,pkt_5_rts,0.0226,0.002675,3.487360e-10,10,0.025349,0.019851
5,pkt_9_ipv4_dst_13,0.0160,0.004522,6.965825e-07,10,0.020647,0.011353
6,pkt_5_ipv4_dst_6,0.0144,0.004971,3.693626e-06,10,0.019509,0.009291
7,pkt_1_rts,0.0132,0.003425,3.376672e-07,10,0.016720,0.009680
8,pkt_2_ipv4_dst_13,0.0108,0.004826,2.905640e-05,10,0.015759,0.005841
9,pkt_9_ipv4_dst_16,0.0096,0.003239,3.057087e-06,10,0.012928,0.006272


In [8]:
column_headers = features_ranks_df.columns.values.tolist()
print("The Column Header :", column_headers)

The Column Header : ['Unnamed: 0', 'importance', 'stddev', 'p_value', 'n', 'p99_high', 'p99_low']


In [9]:
feature_list = features_ranks_df['Unnamed: 0'].tolist()
# print(feature_list)

feature_list_32 = feature_list[:32]
feature_list_128 = feature_list[:128]

print(feature_list_32)

['pkt_9_ipv4_dst_6', 'pkt_2_rts', 'pkt_6_rts', 'pkt_4_rts', 'pkt_5_rts', 'pkt_9_ipv4_dst_13', 'pkt_5_ipv4_dst_6', 'pkt_1_rts', 'pkt_2_ipv4_dst_13', 'pkt_9_ipv4_dst_16', 'pkt_8_rts', 'pkt_9_rts', 'pkt_7_rts', 'pkt_3_rts', 'pkt_8_ipv4_ttl_5', 'pkt_9_ipv4_dst_0', 'pkt_9_ipv4_dst_1', 'pkt_3_tcp_opt_159', 'pkt_1_tcp_wsize_4', 'pkt_0_tcp_opt_64', 'pkt_7_ipv4_dst_0', 'pkt_3_tcp_wsize_4', 'pkt_1_tcp_opt_159', 'pkt_8_ipv4_ttl_1', 'pkt_1_tcp_opt_96', 'pkt_6_ipv4_id_6', 'pkt_4_ipv4_dst_0', 'pkt_1_tcp_seq_8', 'pkt_8_ipv4_dst_7', 'pkt_3_ipv4_dst_1', 'pkt_2_ipv4_dst_7', 'pkt_7_ipv4_src_3']


In [10]:
#select a sub-dataframe with only the 27 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list_32]
df_test = df_test[feature_list_32]

display(df_train)
display(df_test)

,pkt_9_ipv4_dst_6,pkt_2_rts,pkt_6_rts,pkt_4_rts,pkt_5_rts,pkt_9_ipv4_dst_13,pkt_5_ipv4_dst_6,pkt_1_rts,pkt_2_ipv4_dst_13,pkt_9_ipv4_dst_16,...,pkt_1_tcp_opt_159,pkt_8_ipv4_ttl_1,pkt_1_tcp_opt_96,pkt_6_ipv4_id_6,pkt_4_ipv4_dst_0,pkt_1_tcp_seq_8,pkt_8_ipv4_dst_7,pkt_3_ipv4_dst_1,pkt_2_ipv4_dst_7,pkt_7_ipv4_src_3
0,0,564,494,1093,296900,0,0,1615,0,1,...,-1,0,-1,1,0,0,0,1,0,0
1,0,4542,706,1519380,6433,0,0,165,1,1,...,0,1,0,1,0,0,0,1,1,0
2,1,4573,460,68124,1065,0,1,365,1,0,...,-1,0,-1,1,0,1,0,1,1,0
3,1,79,13903,43,31,0,1,2200,1,1,...,1,1,0,0,1,1,1,1,1,0
4,0,29961,193667,2267319,79,1,0,15668,0,0,...,0,0,1,1,0,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,1,40,55,88,50,1,1,86,1,1,...,1,1,0,1,1,1,1,0,1,0
2379,0,161298,803,3327231,1185,0,0,32005,0,1,...,-1,0,-1,1,0,1,0,1,1,0
2380,1,51919,37,49,40,0,1,61,0,0,...,1,1,0,0,0,1,1,1,0,0
2381,0,8129,25414,1263523,103,1,1,265,1,1,...,0,1,0,0,0,0,1,1,1,1


,pkt_9_ipv4_dst_6,pkt_2_rts,pkt_6_rts,pkt_4_rts,pkt_5_rts,pkt_9_ipv4_dst_13,pkt_5_ipv4_dst_6,pkt_1_rts,pkt_2_ipv4_dst_13,pkt_9_ipv4_dst_16,...,pkt_1_tcp_opt_159,pkt_8_ipv4_ttl_1,pkt_1_tcp_opt_96,pkt_6_ipv4_id_6,pkt_4_ipv4_dst_0,pkt_1_tcp_seq_8,pkt_8_ipv4_dst_7,pkt_3_ipv4_dst_1,pkt_2_ipv4_dst_7,pkt_7_ipv4_src_3
0,0,3106089,679639,2974,460,1,0,3522,0,0,...,0,1,1,0,0,0,0,1,0,0
1,0,5433,190,1533214,428,0,0,13952,0,1,...,0,1,1,0,0,0,0,1,0,0
2,0,1116735,186,172,166,0,0,15208,0,1,...,0,1,0,1,0,0,0,1,0,0
3,1,3564227,2270,25,15,0,1,257,0,0,...,1,0,0,0,0,1,1,1,0,0
4,1,3237,13575,1372892,344,1,1,235,1,1,...,0,1,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,533,18678,398,385,1,0,1170,0,0,...,-1,0,-1,0,0,0,0,1,0,0
496,0,274,5,338,4278,1,0,2382,0,1,...,0,0,0,0,0,0,1,1,0,1
497,0,119,61,179,2128,1,1,217,0,0,...,0,1,0,1,0,0,0,0,0,0
498,1,46,61,17930,52,0,1,65,0,0,...,1,1,0,1,1,1,1,1,1,0


In [11]:
hidden_layer_nodes = 32
output_layer_nodes = 4
number_of_attributes = df_train.shape[1]
NUMBER_OF_NN_REPETITIONS = 10
EPOCHS = 500
BATCH_SIZE = 512

In [12]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [13]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Normalize the data
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [16]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8480 - loss: 0.4467
1: train_accuracy:[0.9492236971855164] test_accuracy:[0.8479999899864197]
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
tf.Tensor(
[[ 32   6   4   0]
 [  5 115   7  23]
 [  0   2  70   7]
 [  1  14   7 207]], shape=(4, 4), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.8600 - loss: 0.4179
2: train_accuracy:[0.9492236971855164, 0.9446076154708862] test_accuracy:[0.8479999899864197, 0.8600000143051147]
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
tf.Tensor(
[[ 34   6   1   1]
 [  3 115   8  24]
 [  0   1  71   7]
 [  0  13   6 210]], shape=(4, 4), dtype=int32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.8500 - loss: 0.4359
3: train_accuracy:[0.9492236971855164, 0.9446076154708862, 0.9479647278785706] test_accuracy:[0.8479999899864197, 0.8600000143051147, 0.8500000238418579]
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
tf.Tensor(
[[ 32   5   2   3]
 [  6 114   7  23]
 [  0   2  70   7]
 [  0  14   6 209]], s

In [18]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file

os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-app-iden-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

11-06-2024-15-51-47
C:\NN-P4\tf-model-testing


In [ ]:
# df_train = pd.read_pickle('../datasets/nprint-raw/stream-video-c10-little/X.pkl')
# df_test = pd.read_pickle('../datasets/nprint-raw/stream-video-c10-little/X_val.pkl')
# label_train = pd.read_pickle('../datasets/nprint-raw/stream-video-c10-little/y.pkl')
# label_test = pd.read_pickle('../datasets/nprint-raw/stream-video-c10-little/y_val.pkl')

# df_train.to_csv('../datasets/nprint-raw/stream-video-c10-little/X.csv')
# df_test.to_csv('../datasets/nprint-raw/stream-video-c10-little/X_val.csv')
# label_train.to_csv('../datasets/nprint-raw/stream-video-c10-little/y.csv')
# label_test.to_csv('../datasets/nprint-raw/stream-video-c10-little/y_val.csv')

# display(df_train)
# display(df_test)
# display(label_train)
# display(label_test)